原始论文：XLNet: Generalized Autoregressive Pretraining for Language Understanding
https://arxiv.org/abs/1906.08237

- `AR：Autoregressive Language Modeling`
    - 依据前面（或后面）出现的`tokens`来预测当前时刻的`token`，代表有 `ELMO`， `GPT`等。
    - 它只能利用单向语义而不能同时利用上下文信息。`ELMO` 通过双向都做`AR` 模型，然后进行拼接，但从结果来看，效果并不是太好。
    - 对生成模型友好，天然符合生成式任务的生成过程。这也是为什么 `GPT` 能够编故事的原因
- `AE：Autoencoding Language Modeling`
    - 通过上下文信息来预测被`mask`的`token`，代表有 `BERT` , `Word2Vec(CBOW)`  。
    - 由于训练中采用了 `[MASK]` 标记，导致预训练与微调阶段不一致的问题。BERT独立性假设问题，即没有对被遮掩（`Mask`）的 `token` 之间的关系进行学习。此外对于生成式问题， AE 模型也显得捉襟见肘。
    - 能够很好的编码上下文语义信息（即考虑句子的双向信息）， 在自然语言理解相关的下游任务上表现突出。

`XLNet`结合AR和AE模型的优点，`Permutation Language Model（PLM）`
- 将长度为$T$的序列$x$，重新排列，共有$T!$种排列 --> (如何实现)
    - 如序列$\{1,3,10,54\}$，重排成$\{3,54,10,1\}$
- 从中随机选择一种排列，然后将末尾一定量的词给遮掩掉 --> (如何抽样，如何遮掩)，然后按照`AR`的方式依次预测被遮掩掉的词

![](images/two-stream-attention.PNG)

重新排列(Permutation)具体的实现方式是通过对Attention Mask进行操作，如最右图所示：
序列1234，重排成3241，根据从左至右AR的方式，则1就能看见234，2只能看见3，3为首位看不到其它位置，4可以看见23

为了实现 Permutation 加上 AR 预测过程，首先我们会发现，打乱顺序后位置信息非常重要，同时对每个位置来说，需要预测的是内容信息（对应位置的词），于是输入就不能包含内容信息，不然模型学不到东西，只需要直接从输入复制到输出就好了。

于是这里就造成了位置信息与内容信息的割裂，因此在 BERT 这样的位置信息加内容信息输入 Self-Attention （自注意力） 的流（Stream）之外，作者还增加了另一个只有位置信息作为 Self-Attention 中 Query 输入的流。文中将前者称为 Content Stream，而后者称为 Query Stream。

这样就能利用 Query Stream 在对需要预测位置进行预测的同时，又不会泄露当前位置的内容信息。具体操作就是用两组隐状态 g和 h 。其中  g  只有位置信息，作为 Self-Attention 里的 Q 。h 包含内容信息，则作为 K和V 。具体表示如上图图所示：

第一点，最下面一层蓝色的 Content Stream 的输入是$e(x_i)$，这个很好懂就是$x$ 对应的词向量 (Embedding)，不同词对应不同向量，但看旁边绿色的 Query Stream，就会觉得很奇怪，为什么都是一样的 $w$？这个和Relative Positional Encoding 有关。
第二点，Query stream attention图中为了便于说明，只将当前位置之外的 h
作为K  和 V，但实际上实现中应该是所有时序上的 h 都作为 K 和 V，最后再交给上图中的 Query stream 的 Attention Mask 来完成位置的遮盖。